In [ ]:
#### Implementing AlphaZero algorithm on rubik's cube

In [1]:
import gym
import gym_Rubiks_Cube
import numpy as np

In [2]:
import torch
from torch import nn


In [42]:
import torch.nn.functional as F

class rubiksSolver(nn.Module):
    def __init__(self):
        super(rubiksSolver, self).__init__()
        self.layer_1 = nn.Linear(in_features=54*6, out_features=4096, bias=True)
        self.layer_2 = nn.Linear(in_features=4096, out_features=2048, bias=True)
        self.layer_3 = nn.Linear(in_features=2048, out_features=18+1, bias=True)

    def forward(self, x):
        ## First 3 layers ok
        x = self.layer_1(x)
        x = F.relu(x)
        x = self.layer_2(x)
        x = F.relu(x)
        x = self.layer_3(x)
        
        ## Softmax on the 9 first values
        actions_proba = F.softmax(x[:, :18])
        
        ## relu on the last value
        value_fonction = F.relu(x[:,18])
        
        return actions_proba, value_fonction
        


In [43]:
# now we create the input of the vector
model = rubiksSolver()

# the conversion part 

In [82]:
### Encoders 

from sklearn.preprocessing import OneHotEncoder

lb = OneHotEncoder(sparse=False)
list_type_array = np.array(list(range(6)))
lb.fit(list_type_array.reshape(-1, 1))


lb_action = OneHotEncoder(sparse=False)
list_type_array_action = np.array(list(range(18)))
lb_action.fit(list_type_array_action.reshape(-1, 1))

def input_for_model(obs):
    return lb.transform(obs.reshape(-1, 1)).flatten()

def intput_for_env(action_prob):
    return lb_action.inverse_transform(action)

In [3]:
env = gym.make("RubiksCube-v0")

env.setScramble(1, 2, False)
print(env.reset()) # scramble status
#obs, rew, done, _ = env.step(0)

[5 5 5 5 5 5 5 5 5 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 3
 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4]


In [145]:
#### Main code to explore the possibilities

data_in = input_for_model(obs)
data_in = torch.from_numpy(data_in).float()


action_prob , value_fonction = model(data_in.view(-1, 54*6))

action_choice = np.random.choice(np.array(range(18)), p=action_prob.data.numpy().reshape(9,))

if action_choice < 9:
    obs, rew, done, _ = env.step(action_choice)
else:
    # inverse
    env.step(action_choice)
    env.step(action_choice)
    obs, rew, done, _ = env.step(action_choice)
    


/Users/adrienbufort/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [30]:
def env_next(action_choice, env):
    if action_choice < 9:
        obs, rew, done, _ = env.step(action_choice)
    else:
        # inverse
        env.step(action_choice % 9)
        env.step(action_choice % 9)
        obs, rew, done, _ = env.step(action_choice % 9)
    return obs, rew, done, _ 
        

In [31]:
#### TODO
"""
In the todo we have :

    - The sampling generation x_i for all the shuffle of the cube
    - ADI (see paper) : for each x_i we do a 1-breath for search to find the max of current action

Then we have to code the MCTS to find our solution (or the kind of MCTS) exploring a good chunk of the space to create the solver

"""



'\nIn the todo we have :\n\n    - The sampling generation x_i for all the shuffle of the cube\n    - ADI (see paper) : for each x_i we do a 1-breath for search to find the max of current action\n\nThen we have to code the MCTS to find our solution (or the kind of MCTS) exploring a good chunk of the space to create the solver\n\n'

In [53]:
### Autodidactic value
def train_on_ADI(nb_shuffle = 16, nb_depth_max= 10):
    
    train_set = []
    shuffle = np.random.randint(0, 18, nb_depth_max)
    
    for i in range(nb_shuffle):
        env.reset()
        rew_tmp = 1
        for j in range(nb_depth_max):
            obs, rew, done, _ = env_next(shuffle[j], env)
            train_set.append([input_for_model(obs).reshape((1, 324)), (shuffle[j]+9) % 18, rew_tmp, j+1])
            rew_tmp = rew

    return train_set



In [ ]:
gamma = 0.99

In [89]:
train_set = train_on_ADI()

states = list(map(lambda x: torch.from_numpy(x[0]), train_set))
actions = np.array(list(map(lambda x: x[1], train_set)))




In [ ]:
actions = torch.from_numpy(lb_action.transform(actions.reshape(-1, 1)))

values = torch.tensor(list(map(lambda x: x[2], train_set)))

states = torch.cat(states, 0)
actions = torch.cat(actions, 0)
values = torch.cat(values, 0)


In [90]:
#### Init the model with corrent best stuff we can do
actions.shape

(1000,)

In [19]:
obs, rew, done, _ = env.step(0)
env.render()
print(rew)

          +  +  +                   
          +  +  +                   
          +  +  +                   
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
          +  +  +                   
          +  +  +                   
          +  +  +                   

1.0


In [27]:
env.reset

<bound method RubiksCubeEnv.reset of <gym_Rubiks_Cube.envs.rubiks_cube_env.RubiksCubeEnv object at 0x11c289090>>

In [129]:
action_prob.data.numpy()

array([[0.11916346, 0.10949372, 0.10370624, 0.11654171, 0.11517142,
        0.11497392, 0.10954474, 0.1023467 , 0.10905815]], dtype=float32)

1

In [119]:
data_in.view(-1, 54*6).dim()

2

In [9]:

env.setScramble(1, 10, True)
print(env.reset()) # scramble status
env.render()
obs, rew, done, _ = env.step(0) # action 0~11
obs, rew, done, _ = env.step(1) # action 0~11
env.render()
scramble, action = env.getlog()
print("Scramble: ", scramble)
print("Action: ", action)

[3 1 3 5 5 3 0 0 3 4 2 2 4 2 2 4 1 1 5 1 1 5 1 5 5 1 5 0 0 2 0 0 0 4 2 2 3
 3 2 3 3 2 1 4 1 4 4 0 4 4 3 5 5 0]
          +  +  +                   
          +  +  +                   
          +  +  +                   
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
          +  +  +                   
          +  +  +                   
          +  +  +                   

          +  +  +                   
          +  +  +                   
          +  +  +                   
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  +  +  +  +  + 
          +  +  +                   
          +  +  +                   
          +  +  +                   

Scramble:  [7, 4, 9, 7, 3]
Action:  [0, 1]


In [6]:
print(env.action_space)
print(env.observation_space)


Discrete(12)
Box(54,)


In [20]:
obs, rew, done, _

(array([5, 5, 4, 5, 5, 4, 5, 5, 4, 3, 2, 2, 3, 2, 2, 3, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 5, 3, 3, 5, 3, 3,
        5, 4, 4, 2, 4, 4, 2, 4, 4, 2]), 0.0, False, {})

In [21]:
len(obs)

54

In [5]:


# 2x2 Rubik's Cube
env = gym.make("RubiksCube2x2-v0")

print(env.reset()) # scramble status
env.render()
env.step(0) # action 0~11
env.step(1) # action 0~11
env.render()
scramble, action = env.getlog()
print("Scramble: ", scramble)
print("Action: ", action)

[2 3 5 4 0 5 3 5 0 4 1 1 4 3 4 1 1 2 0 5 3 2 0 2]
       +  +             
       +  +             
 +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  + 
       +  +             
       +  +             

       +  +             
       +  +             
 +  +  +  +  +  +  +  + 
 +  +  +  +  +  +  +  + 
       +  +             
       +  +             

Scramble:  [10, 3, 2, 0, 5, 4, 2]
Action:  [0, 1]


In [82]:


sol = lb_action.transform(np.array(1).reshape(-1, 1))

sol_transform = sol


In [84]:
lb_action.inverse_transform(sol.reshape((1,-1)))

array([[1]])

array([0, 5, 5, 0, 5, 3, 3, 3, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 4, 1, 1,
       1, 1, 5, 5, 1, 0, 0, 4, 0, 0, 4, 4, 2, 0, 3, 3, 3, 3, 3, 1, 2, 0,
       3, 5, 5, 4, 4, 4, 4, 5, 5, 4])

In [57]:
obs

array([0, 5, 5, 0, 5, 3, 3, 3, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 4, 1, 1,
       1, 1, 5, 5, 1, 0, 0, 4, 0, 0, 4, 4, 2, 0, 3, 3, 3, 3, 3, 1, 2, 0,
       3, 5, 5, 4, 4, 4, 4, 5, 5, 4])